In [ ]:
import anndata
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
from scipy.sparse import csr_matrix

from mcDETECT.utils import *
from mcDETECT.model import *
from mcDETECT.downstream import *

import warnings
warnings.filterwarnings("ignore")
sc.settings.verbosity = 0

## ========== Separate mode ==========

In [ ]:
# File paths
dataset = "MERSCOPE_WT_1"
data_path = f"../data/{dataset}/"
output_path = f"../output/{dataset}/"
comparison_path = f"../output/MERSCOPE_WT_AD_comparison/"

if dataset == "MERSCOPE_WT_1":
    flip = True
    cutoff = 6250
    theta = 10 * np.pi / 180
    coordinate_for_rotation = ["sphere_y", "sphere_x"]
    coordinate_for_flip = "global_y"
    granule_markers_plot = ["Camk2a", "Cplx2", "Slc17a7", "Syp", "Ddn", "Map1a", "Syn1", "Shank1", "Cyfip2", "Vamp2"]
    neuron_markers_plot = ["Trpc4", "Gjc3", "Plekhb1", "Ntrk2", "Ntsr2", "Fn1", "Gnai2", "Shc3", "Reep3", "Chd9", "Acsbg1"]
elif dataset == "MERSCOPE_AD_1":
    flip = False
    cutoff = -4200
    theta = 170 * np.pi / 180
    coordinate_for_rotation = ["sphere_x", "sphere_y"]
    coordinate_for_flip = "global_x"
    granule_markers_plot = ["Camk2a", "Cplx2", "Slc17a7", "Syp", "Ddn", "Map1a", "Syn1", "Shank1", "Cyfip2", "Vamp2"]
    neuron_markers_plot = ["Trpc4", "Gjc3", "Plekhb1", "Ntrk2", "Ntsr2", "Fn1", "Gnai2", "Shc3", "Reep3", "Chd9", "Acsbg1"]

In [ ]:
# Colors
color_dct = ["#F56867","#FEB915","#C798EE","#59BE86","#7495D3","#6D1A9C","#15821E","#3A84E6","#997273","#787878","#DB4C6C","#9E7A7A","#554236","#AF5F3C","#93796C","#F9BD3F","#DAB370","#877F6C","#268785"]

In [ ]:
# -------------------- Read data -------------------- #

# Cells
adata = sc.read_h5ad(data_path + "processed_data/adata.h5ad")

# Neurons
adata_neuron = adata[adata.obs["cell_type"].isin(["Glutamatergic", "GABAergic"])].copy()

# Transcripts
transcripts = pd.read_parquet(data_path + "processed_data/transcripts.parquet")

# Genes
genes = pd.read_csv(data_path + "processed_data/genes.csv")
genes = list(genes.iloc[:, 0])

# Negative control markers
nc_genes = pd.read_csv(data_path + "processed_data/negative_controls.csv")
nc_genes = list(nc_genes["Gene"])

# Spots
spots = sc.read_h5ad(data_path + "processed_data/spots.h5ad")

In [ ]:
# Markers
syn_genes = ["Camk2a", "Cplx2", "Slc17a7", "Ddn", "Syp", "Map1a", "Shank1", "Syn1", "Gria1", "Gria2", "Cyfip2", "Vamp2", "Bsn", "Slc32a1", "Nfasc", "Syt1", "Tubb3", "Nav1", "Shank3", "Mapt"]
len(syn_genes)

In [ ]:
# Initialize mcDETECT
mc = mcDETECT(type = "discrete", transcripts = transcripts, gnl_genes = syn_genes, nc_genes = nc_genes, eps = 1.5,
              minspl = 3, grid_len = 1, cutoff_prob = 0.95, alpha = 10, low_bound = 3, size_thr = 4.0,
              in_soma_thr = 0.1, l = 1, rho = 0.2, s = 1, nc_top = 20, nc_thr = 0.1)

In [ ]:
# Read granules
granules = pd.read_parquet(output_path + "granules.parquet") # no "local" for HGCC output
granules.shape

### 1. Spot-level neuron and granule metadata

In [ ]:
# Spot-level neuron and granule metadata
spot_neuron_all = spot_neuron(adata_neuron, spots)
spot_neuron_all.write_h5ad(comparison_path + f"{dataset}_neurons.h5ad")

spot_granule_all = spot_granule(granule = granules, spot = spots)
spot_granule_all.write_h5ad(comparison_path + f"{dataset}_granules.h5ad")

In [ ]:
# Plot spot-level granule metadata
plot_x = "global_" + coordinate_for_rotation[0].split("_")[1] + "_new"
plot_y = "global_" + coordinate_for_rotation[1].split("_")[1] + "_new"

sc.set_figure_params(scanpy = True, figsize = (6, 9))
ax = sc.pl.scatter(spot_granule_all, alpha = 1, x = plot_x, y = plot_y, color = "brain_area", palette = color_dct, size = 40, title = " ", show = False)
ax.grid(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel("")
ax.set_ylabel("")
frame_width = 1.5
for spine in ax.spines.values():
    spine.set_linewidth(frame_width)
plt.show()

### 2. Granule and neuron density

In [ ]:
# Granule and neuron density
granule_array = np.array(spot_granule_all.obs["gnl_count"])
neuron_array = np.array(spot_neuron_all.obs["neuron_count"])
area_array = np.array(spot_granule_all.obs["brain_area"])

granule_density, neuron_density = [], []

area_list = ["Isocortex", "OLF", "HPF-CA", "HPF-DG", "HPF-SR", "CTXsp", "TH", "MB", "FT"]
for j in area_list:
    granule_temp = granule_array[area_array == j]
    neuron_temp = neuron_array[area_array == j]
    granule_density.append(np.sum(granule_temp) / len(granule_temp))
    neuron_density.append(np.sum(neuron_temp) / len(neuron_temp))

density_comparison = pd.DataFrame({"area_list": area_list, "granule_density": granule_density, "neuron_density": neuron_density})
density_comparison.to_csv(comparison_path + f"{dataset}_granule_neuron_density.csv", index = 0)

In [ ]:
# Plot granules
granule_adata = anndata.AnnData(X = np.zeros(granules.shape), obs = granules)
granule_adata.obs["brain_area"] = pd.Categorical(granule_adata.obs["brain_area"], categories = ["CTXsp", "FT", "HPF-CA", "HPF-DG", "HPF-SR", "Isocortex", "MB", "OLF", "TH"], ordered = True)

sc.set_figure_params(scanpy = True, figsize = (6, 9))
ax = sc.pl.scatter(granule_adata, alpha = 1, x = plot_x, y = plot_y, color = "brain_area", palette = color_dct, size = 1, title = " ", show = False)
ax.grid(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel("")
ax.set_ylabel("")
for spine in ax.spines.values():
    spine.set_visible(False)
plt.savefig(output_path + "granules.png", dpi = 300, bbox_inches = "tight")
plt.close()

### 3. Granule and neuron expression heatmap

In [ ]:
# Read granule expression profile
granule_adata = sc.read_h5ad(output_path + "granule_adata_tsne.h5ad")
granule_adata

In [ ]:
# Granule vs neuron expression (nuclei)
np.random.seed(42)
permuted_indices = np.random.permutation(granule_adata.n_obs)
granule_adata_permutated = granule_adata[permuted_indices].copy()

In [ ]:
adata_all = anndata.concat([granule_adata_permutated, adata_neuron], axis = 0, merge = "same")
adata_all.var["genes"] = adata_all.var.index
adata_all.obs["type"] = ["Granules"] * granule_adata_permutated.shape[0] + ["Neurons"] * adata_neuron.shape[0]
adata_all.obs["type"] = pd.Categorical(adata_all.obs["type"], categories = ["Granules", "Neurons"], ordered = True)
adata_all

In [ ]:
gene_groups = {"Granule Markers": granule_markers_plot,
               "Neuron Markers": neuron_markers_plot}
target_genes = [gene for group in gene_groups.values() for gene in group]

var_group_positions = [(0, 10), (10, 20)]
var_group_labels = list(gene_groups.keys())

sc.set_figure_params(scanpy=True, fontsize=10)
fig = sc.pl.heatmap(adata_all, target_genes, groupby = "type", log = True, cmap = "Reds", standard_scale = "var", swap_axes = True, figsize = (10, 4), show = False)
plt.savefig(output_path + "granule_neuron_heatmap.jpeg", dpi = 300, bbox_inches = "tight")
plt.close()

## ========== Combined mode ==========

In [ ]:
# Paths
compared_samples = ["MERSCOPE_WT_1", "MERSCOPE_AD_1"]
data_paths = [f"../data/{dataset}/" for dataset in compared_samples]
output_paths = [f"../output/{dataset}/" for dataset in compared_samples]
comparison_path = "../output/MERSCOPE_WT_AD_comparison/"

# Parameters
if compared_samples == ["MERSCOPE_WT_1", "MERSCOPE_AD_1"]:
    shift_x, shift_y = 12000, 7200
    cutoff = 6250
else:
    raise ValueError("Invalid compared_samples!")

# ==================== Read data ==================== #

# Transcripts
transcripts_1 = pd.read_parquet(data_paths[0] + "processed_data/transcripts.parquet")
transcripts_2 = pd.read_parquet(data_paths[1] + "processed_data/transcripts.parquet")

# Genes
genes = pd.read_csv(data_paths[0] + "processed_data/genes.csv")
genes = list(genes.iloc[:, 0])

# Negative control markers
nc_genes = pd.read_csv(data_paths[0] + "processed_data/negative_controls.csv")
nc_genes = list(nc_genes["Gene"])

# Markers
syn_genes = ["Camk2a", "Cplx2", "Slc17a7", "Ddn", "Syp", "Map1a", "Shank1", "Syn1", "Gria1", "Gria2", "Cyfip2", "Vamp2", "Bsn", "Slc32a1", "Nfasc", "Syt1", "Tubb3", "Nav1", "Shank3", "Mapt"]
print("Number of syn_genes: ", len(syn_genes))

# ==================== Granule expression profile ==================== #

# Read granules
granules_1 = pd.read_parquet(output_paths[0] + "granules.parquet")
granules_2 = pd.read_parquet(output_paths[1] + "granules.parquet")
print("Granules shape: ", granules_1.shape, granules_2.shape)

# Initialize mcDETECT
mc1 = mcDETECT(type = "discrete", transcripts = transcripts_1, gnl_genes = syn_genes, nc_genes = None, eps = 1.5,
               minspl = 3, grid_len = 1, cutoff_prob = 0.95, alpha = 10, low_bound = 3, size_thr = 1e5,
               in_soma_thr = 1.01, l = 1, rho = 0.2, s = 1, nc_top = 20, nc_thr = 0.1)
mc2 = mcDETECT(type = "discrete", transcripts = transcripts_2, gnl_genes = syn_genes, nc_genes = None, eps = 1.5,
               minspl = 3, grid_len = 1, cutoff_prob = 0.95, alpha = 10, low_bound = 3, size_thr = 1e5,
               in_soma_thr = 1.01, l = 1, rho = 0.2, s = 1, nc_top = 20, nc_thr = 0.1)

# Profile granules
granule_adata_1 = mc1.profile(granules_1, genes = genes)
granule_adata_2 = mc2.profile(granules_2, genes = genes)
print("Granule adata shape: ", granule_adata_1.shape, granule_adata_2.shape)

# Add metadata columns
mismatch1 = (granule_adata_1.obs["global_x"].to_numpy() != granules_1["sphere_x"].to_numpy()).sum()
mismatch2 = (granule_adata_2.obs["global_x"].to_numpy() != granules_2["sphere_x"].to_numpy()).sum()

if (mismatch1 > 0) or (mismatch2 > 0):
    raise ValueError(f"Granule metadata and expression profile do not match! mismatches: {mismatch1}, {mismatch2}")
else:
    granule_adata_1.obs["brain_area"] = granules_1["brain_area"].to_numpy()
    granule_adata_1.obs["global_x_new"] = granules_1["global_x_new"].to_numpy()
    granule_adata_1.obs["global_y_new"] = granules_1["global_y_new"].to_numpy()
    granule_adata_2.obs["brain_area"] = granules_2["brain_area"].to_numpy()
    granule_adata_2.obs["global_x_new"] = granules_2["global_x_new"].to_numpy()
    granule_adata_2.obs["global_y_new"] = granules_2["global_y_new"].to_numpy()

# Adjust coordinates (manual)
if compared_samples == ["MERSCOPE_WT_1", "MERSCOPE_AD_1"]:
    granule_adata_1.obs["global_y_new"] = cutoff - granule_adata_1.obs["global_y_new"]

granule_adata_1.obs["global_x_adjusted"] = granule_adata_1.obs["global_y_new"].copy()
granule_adata_1.obs["global_y_adjusted"] = granule_adata_1.obs["global_x_new"].copy()

granule_adata_2.obs["global_x_adjusted"] = granule_adata_2.obs["global_x_new"].copy()
granule_adata_2.obs["global_y_adjusted"] = granule_adata_2.obs["global_y_new"].copy()

granule_adata_2.obs["global_x_adjusted"] += shift_x
granule_adata_2.obs["global_y_adjusted"] += shift_y

# Concatenate granule adata
granule_adata_dict = {compared_samples[0]: granule_adata_1, compared_samples[1]: granule_adata_2}
granule_adata = anndata.concat(granule_adata_dict, axis = 0, merge = "same", label = "batch")
granule_adata.layers["counts"] = csr_matrix(granule_adata.X.copy())

# Normalize granule adata
sc.pp.normalize_total(granule_adata, target_sum=1e4)
sc.pp.log1p(granule_adata)

# t-SNE embedding
sc.tl.pca(granule_adata, n_comps=10, svd_solver="auto")
sc.tl.tsne(granule_adata, n_pcs=10)

# Save granule adata
granule_adata.write_h5ad(comparison_path + "granule_adata_tsne.h5ad")
print("Granule adata: ", granule_adata)